# 📘 DevOps 업무전환 - Story 생성
- **목적**: DevOps 전환을 위해 생성된 에픽에서 스토리 생성, 생성된 스토리의 우선순위 선정
- **입력 데이터**: generate_epic_of_devops.ipynb의 결과물
- **주요 기능**
  - 에픽에서 스토리 생성
  - 스토리 우선 순위 산정
- **작성자**: 김동영
- **작성일**: 2025-08-01

# 0. 환경 설정

In [1]:
import os, sys
sys.path.append(os.path.abspath("../src"))

import dotenv
import json

from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

from story.routes import StoryGeneratorAgent
# from epic.prompts import TASK_TO_EPIC_CONVERTER_PROMPT

/Users/visuworks/Documents/07_n8n/project_manage_agent/src/story/services.py:20: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  self.llm = ChatOpenAI(


In [2]:
# .env 환경번수 load
dotenv.load_dotenv('../.env')

True

# 1. Agent 생성 및 스토리 생성

In [3]:
# 스토리 생성 llm 모델 생성
story_agent = StoryGeneratorAgent(
    openai_api_key = os.getenv("OPENAI_API_KEY")
)

In [4]:
STORY_GENERATOR_PROMPT = ChatPromptTemplate.from_messages(
    [SystemMessagePromptTemplate.from_template(
        """
        [Role] 너는 숙련된 스크럼 마스터다.
        [Goal] 2주 단위 스프린트를 기반으로 한 애자일 방법론에서, 프로젝트 정보와 Epic에 따라 실제로 필요한 Story를 도출하고 업무 계획을 수립하는 데 도움을 주는 것.
        [Instruction] 주어진 프로젝트 정보와 Epic을 기반으로, Epic를 구성하는 하나 이상의 Story을 생성해.
        [Constraints]
          - 출력은 반드시 [Output Format]을 따라야 한다.
          - Story는 하나의 스프린트 기간 내에 완료될 수 있는 작업 단위여야 한다.
          - 불필요한 설명, 주석, 마크다운 없이 JSON만 출력한다.
          - 최대 {max_storys}개의 Story을 생성한다.
          - 해당 에픽을 완수하기 위한 최소한의 Story만을 생성한다.
          - acceptance_criteria는 구체적으고 측정 가능한 기준으로 작성한다.
          - 에픽에 테스트에 대한 내용이 없으면 테스트 항목은 제외한다.
          - domain은 해당 스토리의 주요 개발 영역을 명시한다.
          - story_type은 스토리의 성격을 분류한다.
          - tags는 관련 기술스택, 기능영역, 키워드를 포함한다.
        [Process]
        1. Epic 분석: 주어진 Epic의 범위와 목표를 파악한다
        2. Story 도출: Epic을 완료하기 위해 필요한 독립적인 작업 단위들을 식별한다
        3. 우선순위 결정: 비즈니스 가치와 의존성을 고려하여 우선순위를 설정한다
        4. 수락 기준 정의: 각 Story의 완료 조건을 구체적으로 명시한다
        [Output Format]
        [
          {{
            "title": "스토리 제목",
            "description": "스토리 설명", 
            "acceptance_criteria": ["수락 기준 1", "수락 기준 2", "수락 기준 3"],
            "domain":  "frontend|backend|devops|data",
            "story_type": "feature|bug|refactor|enhancement|documentation",
            "tags": ["관련 키워드", "기술스택", "기능영역"]
          }},
        ]
        """
    ),
    HumanMessagePromptTemplate.from_template(
        """
        [Input] {user_input}
        [Context] {epic_info}
        """
    )
    ]
)


In [ ]:
# epic_info
epic_info = {
    "title": "CI/CD 자동화 구성",
    "description": "GitHub Actions 또는 Jenkins를 활용하여 End-to-End CI/CD 자동화 프로세스를 구성한다.",
    "business_value": "개발 및 배포 프로세스의 효율성을 높이고, 코드 품질을 유지하여 빠른 피드백을 제공한다.",
    "priority": "High",
    "acceptance_criteria": [
      "PR 생성 시 Lint, Unit Test, Build, Code Quality Check가 자동으로 실행된다.",
      "Dockerfile 실행 오류 및 종속성 검증이 포함된다.",
      "CI/CD 결과가 Slack/이메일/웹훅으로 알림된다."
    ],
    "included_tasks": [
      "GitHub Actions 또는 Jenkins를 활용한 End-to-End 자동화 구성",
      "PR 생성 시 Lint, Unit Test, Build, Code Quality Check 자동 실행",
      "Dockerfile 실행 오류 및 종속성 검증 포함",
      "CI/CD 결과를 Slack/이메일/웹훅 알림으로 연동"
    ]
  },

In [ ]:
generated_storys_str = story_agent._generate_storys_with_llm(
    prompt = STORY_GENERATOR_PROMPT,
    user_input = '',
    epic_info= epic_info,
    max_storys=10
)

In [ ]:
parsed = json.loads(generated_storys_str)
parsed

# 에픽별 스토리 산출

In [11]:
epics_list = [{
    "title": "CI/CD 자동화 구성",
    "description": "GitHub Actions 또는 Jenkins를 활용하여 End-to-End CI/CD 자동화 프로세스를 구성한다.",
    "business_value": "개발 및 배포 프로세스의 효율성을 높이고, 코드 품질을 유지하여 빠른 피드백을 제공한다.",
    "priority": "High",
    "acceptance_criteria": [
      "PR 생성 시 Lint, Unit Test, Build, Code Quality Check가 자동으로 실행된다.",
      "Dockerfile 실행 오류 및 종속성 검증이 포함된다.",
      "CI/CD 결과가 Slack/이메일/웹훅으로 알림된다."
    ],
    "included_tasks": [
      "GitHub Actions 또는 Jenkins를 활용한 End-to-End 자동화 구성",
      "PR 생성 시 Lint, Unit Test, Build, Code Quality Check 자동 실행",
      "Dockerfile 실행 오류 및 종속성 검증 포함",
      "CI/CD 결과를 Slack/이메일/웹훅 알림으로 연동"
    ]
  },{
	  "title": "브랜치 보호 및 환경/비밀 관리",
	  "description": "코드 품질을 보장하기 위한 브랜치 보호 설정과, 각 환경별 구성 및 민감 정보의 안전한 관리를 수행한다.",
	  "business_value": "협업 효율성과 코드 안정성을 높이고, 환경별 배포 신뢰성을 강화한다.",
	  "priority": "Medium",
	  "acceptance_criteria": [
	    "PR 병합은 main, develop 브랜치로 제한된다.",
	    "브랜치 보호 규칙이 적용되어 있다.",
	    "각 브랜치/환경에 대해 별도 설정 파일 및 비밀 저장소가 관리된다."
	  ],
	  "included_tasks": [
	    "main, develop 브랜치에 대한 병합 제한 규칙 설정",
	    "브랜치 보호 규칙(리뷰 승인, 상태 확인 등) 적용",
	    ".env 파일, Secrets Manager, Parameter Store를 통한 환경 및 비밀 설정 구성",
	    "환경별 배포 구성 및 변수 관리 문서화"
	  ]
 },{
    "title": "Docker 이미지 관리",
    "description": "CI/CD 프로세스에서 Docker 이미지를 자동으로 빌드하고 푸시하는 기능을 구현하고, 이미지 생성 및 관리 방안을 정립한다.",
    "business_value": "애플리케이션 배포의 일관성을 유지하고, 배포 속도를 향상시킨다.",
    "priority": "High",
    "acceptance_criteria": [
      "CodeBuild와 연동하여 buildspec.yml이 사용된다.",
      "DockerHub 또는 ECR에 결과 이미지가 자동으로 푸시된다."
    ],
    "included_tasks": [
      "어플리케이션 이미지 단위 패키징 정책 논의",
      "DockerHub 또는 ECR에 결과 이미지 자동 푸시",
      "Docker Image Build 자동화",
      "Docker image build 시 테스트 방안 논의"
    ]
  },
  {
  "title": "배포 전략 및 운영 안정성 관리",
  "description": "다양한 배포 전략, 보안 설정, 헬스체크 및 롤백을 통합하여 운영 중단 없이 안전하고 신뢰성 있는 배포를 지원한다.",
  "business_value": "서비스 중단 없는 배포를 가능하게 하여 사용자 경험을 보호하고, 운영 리스크를 최소화한다.",
  "priority": "High",
  "acceptance_criteria": [
    "배포 트리거가 GitHub Webhook, PR 병합, 수동 승인 등으로 설정된다.",
    "배포 전략이 블루그린, 카나리아, 롤링 중 하나 이상으로 설정된다.",
    "AWS IAM Role 및 SSH 키를 통한 보안 배포가 구성된다.",
    "배포 전후 헬스체크가 수행되고, 실패 시 자동 롤백이 작동한다."
  ],
  "included_tasks": [
    "배포 트리거 전략 수립 및 적용(GitHub Webhook, 수동 승인 등)",
    "배포 전략 적용(블루그린, 카나리아, 롤링 중 선택)",
    "AWS IAM Role 및 SSH 키를 통한 안전한 배포 구성",
    "배포 전/후 헬스체크 구현 및 실패 시 자동 롤백 설정",
    "배포 전략 및 롤백 구성 관련 문서화"
  ]
},
  {
  "title": "운영 정책 및 배포 이력 관리",
  "description": "배포 정책과 템플릿을 표준화하고, 배포 이력 및 승인 기록을 체계적으로 관리하여 프로젝트의 일관성과 추적 가능성을 확보한다.",
  "business_value": "운영 프로세스를 표준화하여 품질을 유지하고, 문제 발생 시 신속한 원인 파악 및 대응이 가능하도록 한다.",
  "priority": "Medium",
  "acceptance_criteria": [
    "배포 이력 및 승인 내역이 저장되고 관리된다.",
    "워크플로 및 파이프라인의 명명 규칙이 명확히 정의되어 있다.",
    "신규 프로젝트에 사용할 수 있는 배포 정책 템플릿과 체크리스트가 제공된다."
  ],
  "included_tasks": [
    "배포 이력 및 승인 기록 저장 시스템 구성",
    "운영 및 배포 프로세스에 대한 정책 문서화",
    "워크플로 및 파이프라인 명명 규칙 정의",
    "신규 프로젝트용 배포 템플릿 및 체크리스트 작성 및 배포"
  ]
}]

In [12]:
storys_by_epic = {}

for epic in epics_list[1:]:
    storys_by_epic[epic['title']] = {}

    generated_storys_str = story_agent._generate_storys_with_llm(
        prompt = STORY_GENERATOR_PROMPT,
        user_input = '',
        epic_info= epic,
        max_storys=10
    )
    parsed = json.loads(generated_storys_str)
    storys_by_epic[epic['title']] = parsed


INFO:story.services:스토리 생성 시작
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:story.services:스토리 생성 완료
INFO:story.services:스토리 생성 시작
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:story.services:스토리 생성 완료
INFO:story.services:스토리 생성 시작
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:story.services:스토리 생성 완료
INFO:story.services:스토리 생성 시작
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:story.services:스토리 생성 완료


In [15]:
storys_by_epic['운영 정책 및 배포 이력 관리']

[{'title': '배포 이력 및 승인 기록 저장 시스템 구성',
  'description': '배포 이력과 승인 기록을 저장하고 관리할 수 있는 시스템을 구성한다.',
  'acceptance_criteria': ['배포 이력과 승인 기록이 데이터베이스에 저장된다.',
   '저장된 이력은 조회 및 필터링이 가능하다.',
   '시스템이 배포 이력의 자동 기록 기능을 제공한다.'],
  'domain': 'backend',
  'story_type': 'feature',
  'tags': ['배포 관리', '데이터베이스', '운영 프로세스']},
 {'title': '운영 및 배포 프로세스에 대한 정책 문서화',
  'description': '운영 및 배포 프로세스에 대한 정책을 문서화하여 팀원들이 쉽게 접근할 수 있도록 한다.',
  'acceptance_criteria': ['정책 문서가 팀의 위키 또는 문서 저장소에 업로드된다.',
   '문서에는 배포 절차와 승인 프로세스가 포함된다.',
   '문서가 팀원들에게 공유되고 피드백을 받는다.'],
  'domain': 'documentation',
  'story_type': 'documentation',
  'tags': ['문서화', '운영 정책', '배포 프로세스']},
 {'title': '워크플로 및 파이프라인 명명 규칙 정의',
  'description': '배포 워크플로 및 파이프라인의 명명 규칙을 정의하여 일관성을 유지한다.',
  'acceptance_criteria': ['명명 규칙이 문서화되어 팀원들에게 배포된다.',
   '명명 규칙이 실제 배포 파이프라인에 적용된다.',
   '팀원들이 명명 규칙을 준수하고 있는지 검토한다.'],
  'domain': 'devops',
  'story_type': 'feature',
  'tags': ['명명 규칙', 'CI/CD', '운영 프로세스']},
 {'title': '신규 프로젝트용 배포 템플릿 및 체크리스트 작성',
  'des